In [ ]:
!pip install mistral_inference
!pip install huggingface_hub

In [1]:
from huggingface_hub import snapshot_download, login
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', 'Nemo-Instruct')
mistral_models_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="mistralai/Mistral-Nemo-Instruct-2407", allow_patterns=["params.json", "consolidated.safetensors", "tekken.json"], local_dir=mistral_models_path)

In [2]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

# tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tekken.json")
# model = Transformer.from_folder(mistral_models_path)

# prompt = "How expensive would it be to ask a window cleaner to clean all windows in Paris. Make a reasonable guess in US Dollar."

# completion_request = ChatCompletionRequest(messages=[UserMessage(content=prompt)])

# tokens = tokenizer.encode_chat_completion(completion_request).tokens

# out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.35, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
# result = tokenizer.decode(out_tokens[0])

# print(result)


In [3]:
class MistralChatbot:
  def __init__(self):
    self.tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tekken.json")
    self.model = Transformer.from_folder(mistral_models_path)
    self.history = []

  def generate_response(self, user_input):

    completion_request = ChatCompletionRequest(messages=[UserMessage(content=user_input)])
    tokens = self.tokenizer.encode_chat_completion(completion_request).tokens

    out_tokens, _ = generate([tokens], self.model, max_tokens=64, temperature=0.35, eos_id=self.tokenizer.instruct_tokenizer.tokenizer.eos_id)
    result = self.tokenizer.decode(out_tokens[0])

    # Update history
    self.history.append({"role": "user", "content": user_input})
    self.history.append({"role": "assistant", "content": result})

    return result

  def count_tokens(self, user_input):
      input_tokens = self.tokenizer(user_input, return_tensors="pt")["input_ids"]
      print("Token count:", input_tokens.shape[1])


In [4]:
mistralChat = MistralChatbot()

Dropping extra args {'max_position_embeddings': 131072}


In [7]:
import json
import random

prompts = json.load(open('prompts_list.json'))
# choose k random prompts
random_prompts = random.choices(prompts, k=20)

In [8]:
for prompt in random_prompts:
	mistralChat.generate_response(prompt)

'lllyasviel/control_v11p_sd15_openpose'

'runwayml/stable-diffusion-v1-5'

'camenduru/text2-video-zero'

'lllyasviel/sd-controlnet-canny'

'lllyasviel/control_v11p_sd15_lineart'

...

'lllyasviel/control_v11p_sd15_depth'

'camenduru/text2-video-zero'

'none'

'lllyasviel/sd-controlnet-canny'

'none'

In [24]:
with open('random_prompts.json', 'w', encoding='utf-8') as f:
    json.dump(random_prompts, f, ensure_ascii=False, indent=2)